In [1]:
!pip -q install transformers==4.17
!pip -q install optuna
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 48.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9

In [2]:
from transformers import Trainer, TrainingArguments, AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import optuna
import time

In [3]:
splits = {'java_train': 'data/java_train-00000-of-00001.parquet', 'java_test': 'data/java_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["java_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["java_test"])

display(train_df.head())
display(test_df.head())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,index,class,comment_sentence,partition,combo,labels
0,0,Abfss.java,azure blob file system implementation of abstr...,0,azure blob file system implementation of abstr...,"[1, 0, 0, 0, 0, 0, 0]"
1,1,Abfss.java,this impl delegates to the old filesystem,0,this impl delegates to the old filesystem | Ab...,"[0, 0, 1, 0, 0, 0, 0]"
2,2,AbstractContractGetFileStatusTest.java,test getfilestatus and related listing operati...,0,test getfilestatus and related listing operati...,"[1, 0, 0, 0, 0, 0, 0]"
3,11,AbstractContractGetFileStatusTest.java,path filter which only expects paths whose fin...,0,path filter which only expects paths whose fin...,"[0, 0, 1, 0, 0, 0, 0]"
4,15,AbstractContractGetFileStatusTest.java,a filesystem filter which exposes the protecte...,0,a filesystem filter which exposes the protecte...,"[0, 0, 1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,5,AbstractContractGetFileStatusTest.java,accept everything.,1,accept everything. | AbstractContractGetFileSt...,"[0, 0, 1, 0, 0, 0, 0]"
1,8,AbstractContractGetFileStatusTest.java,accept nothing.,1,accept nothing. | AbstractContractGetFileStatu...,"[0, 0, 1, 0, 0, 0, 0]"
2,12,AbstractContractGetFileStatusTest.java,equals the @code match field.,1,equals the @code match field. | AbstractContra...,"[0, 0, 1, 0, 0, 0, 0]"
3,17,AbstractS3ACommitterFactory.java,dynamically create the output committer based ...,1,dynamically create the output committer based ...,"[0, 0, 1, 0, 0, 0, 0]"
4,19,ApplicationConstants.java,environment for applications.,1,environment for applications. | ApplicationCon...,"[0, 0, 1, 0, 0, 0, 0]"


In [4]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

# Apply the function to the 'combo' column
train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-4-df4392ae2e99>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-4-df4392ae2e99>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,0,abfss.java,azure blob file system implementation of abstr...,0,azure blob file system implementation of abstr...,"[1, 0, 0, 0, 0, 0, 0]"
1,1,abfss.java,this impl delegates to the old filesystem,0,this impl delegates to the old filesystem | ab...,"[0, 0, 1, 0, 0, 0, 0]"
2,2,abstractcontractgetfilestatustest.java,test getfilestatus and related listing operati...,0,test getfilestatus and related listing operati...,"[1, 0, 0, 0, 0, 0, 0]"
3,11,abstractcontractgetfilestatustest.java,path filter which only expects paths whose fin...,0,path filter which only expects paths whose fin...,"[0, 0, 1, 0, 0, 0, 0]"
4,15,abstractcontractgetfilestatustest.java,a filesystem filter which exposes the protecte...,0,a filesystem filter which exposes the protecte...,"[0, 0, 1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,5,abstractcontractgetfilestatustest.java,accept everything.,1,accept everything | abstractcontractgetfilesta...,"[0, 0, 1, 0, 0, 0, 0]"
1,8,abstractcontractgetfilestatustest.java,accept nothing.,1,accept nothing | abstractcontractgetfilestatus...,"[0, 0, 1, 0, 0, 0, 0]"
2,12,abstractcontractgetfilestatustest.java,equals the @code match field.,1,equals the code match field | abstractcontract...,"[0, 0, 1, 0, 0, 0, 0]"
3,17,abstracts3acommitterfactory.java,dynamically create the output committer based ...,1,dynamically create the output committer based ...,"[0, 0, 1, 0, 0, 0, 0]"
4,19,applicationconstants.java,environment for applications.,1,environment for applications | applicationcons...,"[0, 0, 1, 0, 0, 0, 0]"


In [5]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [6]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/codebert-base')
model = AutoModelForSequenceClassification.from_pretrained('microsoft/codebert-base', num_labels=7)

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [7]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [8]:
class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # Remove 'labels' from inputs before passing to model
        model_inputs = {k: v for k, v in inputs.items() if k != "labels"}
        outputs = model(**model_inputs)  # Pass only the required inputs to the model
        logits = outputs.get("logits")

        if logits is None:
            # Handle the case where logits are missing, e.g., raise an exception or return a default loss
            raise ValueError("Logits are missing from the model output.")  # Or return a default loss

        # Use BCEWithLogitsLoss for multi-label classification
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        return (loss, outputs) if return_outputs else loss

In [9]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)

    labels = pred.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = (preds == labels).mean()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [10]:
def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size * 2,
        warmup_steps=10,
        weight_decay=weight_decay,
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        gradient_accumulation_steps=2,
        report_to="none",
    )

    trainer = OneHotTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_results = trainer.evaluate()

    return eval_results["eval_f1"]

In [11]:






































study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("Best trial:")
print(study.best_trial)

[I 2024-12-11 06:25:27,242] A new study created in memory with name: no-name-a9ca3b5a-eaaf-4587-bb96-78df49473426
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=Tru

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,0.414200,0.248925,0.925242,0.698643,0.673469,0.663937
1,0.218200,0.189359,0.946253,0.773066,0.743622,0.755689
2,0.190400,0.166773,0.953288,0.777712,0.792092,0.782919
3,0.162400,0.153845,0.955257,0.783761,0.797194,0.789451
4,0.152000,0.147263,0.957040,0.790162,0.802934,0.795457
5,0.140900,0.142519,0.958071,0.796049,0.802296,0.798619
6,0.138100,0.139929,0.958916,0.799538,0.804209,0.801458
7,0.135600,0.139465,0.958634,0.799494,0.802934,0.800718


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 8
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 8
/us

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2024-12-11 06:38:38,653] Trial 0 finished with value: 0.800718456755237 and parameters: {'learning_rate': 1.7464990990722544e-06, 'batch_size': 4, 'num_train_epochs': 8, 'weight_decay': 0.0002001260258542116}. Best is trial 0 with value: 0.800718456755237.
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This featur

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.131060,0.959103,0.833586,0.805485,0.811150
2,0.128300,0.125465,0.959572,0.827561,0.819515,0.819661
3,0.117400,0.125831,0.960510,0.835717,0.830357,0.829088
4,0.107200,0.122669,0.960041,0.847347,0.829719,0.837158
5,0.107200,0.118690,0.962480,0.853475,0.839923,0.845540
6,0.103200,0.116571,0.962386,0.851826,0.836097,0.841815
7,0.097200,0.116727,0.961354,0.872471,0.834184,0.843940
8,0.095900,0.116364,0.962761,0.868021,0.841199,0.846826
9,0.095900,0.115760,0.962011,0.858296,0.840561,0.845848


...

[I 2024-12-11 08:01:36,945] Trial 8 finished with value: 0.9007638444292679 and parameters: {'learning_rate': 8.712934557175608e-06, 'batch_size': 16, 'num_train_epochs': 8, 'weight_decay': 0.007536128775423188}. Best is trial 6 with value: 0.901346889999532.
<ipython-input-10-9265609bdcc0>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-9265609bdcc0>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)
PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: Th

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.185145,0.971203,0.903866,0.896046,0.899450
2,0.000600,0.187560,0.970922,0.901205,0.896684,0.898322
3,0.000800,0.190401,0.970828,0.898537,0.897959,0.897505
4,0.000700,0.190995,0.971203,0.900759,0.897959,0.898688
5,0.000700,0.190047,0.970828,0.899308,0.897321,0.897747
6,0.001000,0.189119,0.971203,0.901566,0.897959,0.899297
7,0.000700,0.189065,0.971391,0.902227,0.898597,0.899965
8,0.000600,0.188920,0.971391,0.902227,0.898597,0.899965


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/con

[I 2024-12-11 08:12:34,267] Trial 9 finished with value: 0.8999648798937144 and parameters: {'learning_rate': 1.338909886608613e-06, 'batch_size': 8, 'num_train_epochs': 8, 'weight_decay': 0.006679342884962747}. Best is trial 6 with value: 0.901346889999532.


Best trial:
FrozenTrial(number=6, state=TrialState.COMPLETE, values=[0.901346889999532], datetime_start=datetime.datetime(2024, 12, 11, 7, 24, 4, 943877), datetime_complete=datetime.datetime(2024, 12, 11, 7, 35, 11, 689506), params={'learning_rate': 4.7571390769145025e-05, 'batch_size': 16, 'num_train_epochs': 9, 'weight_decay': 0.0012213826960895207}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=5e-05, log=True, low=1e-06, step=None), 'batch_size': CategoricalDistribution(choices=(4, 8, 16)), 'num_train_epochs': IntDistribution(high=10, log=False, low=3, step=1), 'weight_decay': FloatDistribution(high=0.1, log=True, low=0.0001, step=None)}, trial_id=6, value=None)


In [12]:
best_hyperparams = study.best_trial.params

training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=best_hyperparams["num_train_epochs"],
    per_device_train_batch_size=best_hyperparams["batch_size"],
    per_device_eval_batch_size=best_hyperparams["batch_size"] * 2,
    learning_rate=best_hyperparams["learning_rate"],
    weight_decay=best_hyperparams["weight_decay"],
    warmup_steps=10,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,
    report_to="none",
)

trainer = OneHotTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

start_time = time.time()
trainer.train()
end_time = time.time()
training_time = end_time - start_time
print("<<=================Training Time================>>")
print(f"Training Time: {training_time:.2f} seconds")
print("<<==============================================>>")

PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6091
  Num Epochs = 9
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 1710


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.183450,0.966889,0.894925,0.880102,0.887137
1,No log,0.184939,0.969327,0.898524,0.890944,0.889967
2,0.013900,0.186617,0.969890,0.898024,0.897321,0.895830
3,0.013900,0.185791,0.970828,0.901237,0.893495,0.896332
4,0.013900,0.185428,0.971860,0.904599,0.898597,0.900968
5,0.005300,0.196520,0.968296,0.898984,0.893495,0.893733
6,0.005300,0.184398,0.969890,0.897899,0.896684,0.896747
7,0.002000,0.187403,0.971954,0.904789,0.898597,0.900550
8,0.002000,0.191822,0.971110,0.903202,0.897321,0.899142


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
Saving model checkpoint to ./best_results/checkpoint-500
Configuration saved in ./best_results/checkpoint-500/config.json
Model weights saved in ./best_results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
Saving model checkpoint to ./best_results/checkpoint-1000
Configuration saved in ./best_results/checkpoint-1000/config.json
Model weights saved in ./best_results/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32
Saving model checkpoint to ./best_results/checkpoint-1500
Configuration saved in ./best_results/checkpoint-1500/co

<<=================Training Time================>>
Training Time: 657.26 seconds
<<==============================================>>


In [20]:
eval_results = trainer.evaluate()
print(eval_results)

***** Running Evaluation *****
  Num examples = 1523
  Batch size = 32


{'eval_loss': 0.19182150065898895, 'eval_accuracy': 0.9711096520026264, 'eval_precision': 0.903202336248522, 'eval_recall': 0.8973214285714286, 'eval_f1': 0.8991416361272727, 'eval_runtime': 5.4674, 'eval_samples_per_second': 278.558, 'eval_steps_per_second': 8.779, 'epoch': 9.0}


### Model Saving

In [21]:
!mkdir 'best_model_codebert-base_java'
!mkdir 'best_model_codebert-base_java_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_codebert-base_java')
tokenizer.save_pretrained('./best_model_codebert-base_java_tokenizer')

# Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('./best_model_codebert-base_java')
tokenizer = AutoTokenizer.from_pretrained('./best_model_codebert-base_java_tokenizer')

mkdir: cannot create directory ‘best_model_codebert-base_java’: File exists
mkdir: cannot create directory ‘best_model_codebert-base_java_tokenizer’: File exists


Saving model checkpoint to ./best_model_codebert-base_java
Configuration saved in ./best_model_codebert-base_java/config.json
Model weights saved in ./best_model_codebert-base_java/pytorch_model.bin
tokenizer config file saved in ./best_model_codebert-base_java_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_codebert-base_java_tokenizer/special_tokens_map.json
loading configuration file ./best_model_codebert-base_java/config.json
Model config RobertaConfig {
  "_name_or_path": "./best_model_codebert-base_java",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_

### Model Testing

In [22]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [23]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

In [24]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

# Measure inference time
start_time = time.time()

# Predict on the test dataset
predictions = trainer.predict(test_dataset)

# Calculate elapsed time
end_time = time.time()
inference_time = end_time - start_time

# Compute metrics using the `compute_metrics` function
metrics = compute_metrics(predictions)

print("<<=================Evaluation Metrics================>>")
print("Evaluation Metrics:", metrics)
print("<<==============================================>>")
print("<<=================Inference Time================>>")
print(f"Inference Time: {inference_time:.2f} seconds")
print("<<==============================================>>")

***** Running Prediction *****
  Num examples = 1725
  Batch size = 32


<<=================Evaluation Metrics================>>
Evaluation Metrics: {'accuracy': 0.9414492753623188, 'precision': 0.8117680494057317, 'recall': 0.7973517559009787, 'f1': 0.8018167641122924}
<<==============================================>>
<<=================Inference Time================>>
Inference Time: 6.24 seconds
<<==============================================>>


In [25]:
!zip -r best_model_codebert-base_java.zip './best_model_codebert-base_java'
!zip -r best_model_codebert-base_java_tokenizer.zip './best_model_codebert-base_java_tokenizer'

updating: best_model_codebert-base_java/ (stored 0%)
updating: best_model_codebert-base_java/pytorch_model.bin (deflated 7%)
updating: best_model_codebert-base_java/config.json (deflated 55%)
updating: best_model_codebert-base_java/training_args.bin (deflated 51%)
updating: best_model_codebert-base_java_tokenizer/ (stored 0%)
updating: best_model_codebert-base_java_tokenizer/merges.txt (deflated 53%)
updating: best_model_codebert-base_java_tokenizer/tokenizer.json (deflated 82%)
updating: best_model_codebert-base_java_tokenizer/vocab.json (deflated 59%)
updating: best_model_codebert-base_java_tokenizer/special_tokens_map.json (deflated 51%)
updating: best_model_codebert-base_java_tokenizer/tokenizer_config.json (deflated 73%)


In [26]:
# Transferring the model to Google Drive
from google.colab import drive
drive.mount('/content/drive')

!cp best_model_codebert-base_java.zip "/content/drive/MyDrive/FYP/Revised/Models/codebert-base_java/"
!cp best_model_codebert-base_java_tokenizer.zip "/content/drive/MyDrive/FYP/Revised/Models/codebert-base_java_Tokenizer/"

Mounted at /content/drive


### Load and Test Model

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Replace 'path/to/checkpoint-folder' with the actual path to your checkpoint folder.
model_folder = '/content/drive/MyDrive/FYP/Revised/Models/codebert-base_java/'
tokenizer_folder = '/content/drive/MyDrive/FYP/Revised/Models/codebert-base_java_Tokenizer/'

!unzip '/content/drive/MyDrive/FYP/Revised/Models/codebert-base_java/best_model_codebert-base_java.zip' -d './'
!unzip '/content/drive/MyDrive/FYP/Revised/Models/codebert-base_java_Tokenizer/best_model_codebert-base_java_tokenizer.zip' -d './'

Archive:  /content/drive/MyDrive/FYP/Revised/Models/codebert-base_java/best_model_codebert-base_java.zip
replace ./best_model_codebert-base_java/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_codebert-base_java/pytorch_model.bin  
  inflating: ./best_model_codebert-base_java/config.json  
  inflating: ./best_model_codebert-base_java/training_args.bin  
Archive:  /content/drive/MyDrive/FYP/Revised/Models/codebert-base_java_Tokenizer/best_model_codebert-base_java_tokenizer.zip
replace ./best_model_codebert-base_java_tokenizer/merges.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./best_model_codebert-base_java_tokenizer/merges.txt  
  inflating: ./best_model_codebert-base_java_tokenizer/tokenizer.json  
  inflating: ./best_model_codebert-base_java_tokenizer/vocab.json  
  inflating: ./best_model_codebert-base_java_tokenizer/special_tokens_map.json  
  inflating: ./best_model_codebert-base_java_tokenizer/tokenizer_config.json  


In [29]:
best_model_codebert_base_java = 'best_model_codebert-base_java'
best_model_codebert_base_java_tokenizer = 'best_model_codebert-base_java_tokenizer'

In [30]:
# Load the model and tokenizer from the checkpoint
model = AutoModelForSequenceClassification.from_pretrained(best_model_codebert_base_java)
tokenizer = AutoTokenizer.from_pretrained(best_model_codebert_base_java_tokenizer)

loading configuration file best_model_codebert-base_java/config.json
Model config RobertaConfig {
  "_name_or_path": "best_model_codebert-base_java",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "to

In [31]:
text = "for detecting automatically generated fields."
start_time = time.time()
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
end_time = time.time()
inference_time = end_time - start_time
print("===============Inference Time===============")
print(f"Inference Time: {inference_time:.2f} seconds")
print("============================================")
output

===============Inference Time===============
Inference Time: 0.25 seconds


SequenceClassifierOutput(loss=None, logits=tensor([[  2.9100, -12.2875,   0.3169,  -6.3027,  -9.6951, -10.7222, -10.5175]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [32]:
logits = output.logits

# Apply sigmoid to convert logits to probabilities
probabilities = torch.sigmoid(logits)

# Define a threshold to determine if a class is positive
threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

# Convert indices to a list for output
predicted_classes = (predicted_indices + 1).tolist()

# Format the output to display classes
if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")

Predicted class/es: 1, 3
